<a href="https://colab.research.google.com/github/LotusBro98/Lanat2019/blob/master/Machine_Learning_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинное обучение
## Генеративные состязательные сети

---

<img width="100%" src="https://pa1.narvii.com/6910/9641f0ed17e10ad698e4e9c23a3de25550339fc5r1-811-397_hq.gif">

---

В данном примере будет рассмотрена архитектура DCGAN для генерации Pixel Art персонажей

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import glob
import random

import tensorflow.keras.layers as layers

In [0]:
SIZE = 64
MAX_IMAGES = 1000
OUTPUT_CHANNELS = 4
N_INPUT_PARAMS = 512

## Загрузим и подготовим датасет

In [0]:
from google.colab import drive
drive.mount('/content/drive')

app_dir = "/content/drive/My Drive/pixelartgen"
!mkdir -p "{app_dir}"
os.chdir(app_dir)

img_dir = "images"
!mkdir -p "{img_dir}"

In [0]:
def load_image(filename):
  image = tf.io.read_file(filename)
  image = tf.io.decode_image(image, channels=4)

  image = tf.image.resize(image, (SIZE, SIZE), method='nearest')
  image = tf.cast(image, tf.float32)
  image = image / 127.5 - 1.0

  return image

In [0]:
files = glob.glob(os.path.join(img_dir, "*.*"))
files = files[:MAX_IMAGES]
train_images = list(map(load_image, files))
train_images = tf.stack(train_images)

print(train_images.shape)

### Отобразим несколько картинок из датасета

In [0]:
N_PLOTS = 8
plt.figure(figsize=(10, 10))
for i in range(N_PLOTS * N_PLOTS):
  plt.subplot(N_PLOTS, N_PLOTS, i + 1)
  plt.imshow(random.choice(train_images) * 0.5 + 0.5)
  plt.axis('off')
plt.show()

## Определим модель генератора

In [0]:
def upsample(x, filters, size):
    x = layers.Conv2DTranspose(filters, (size, size), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    return x

In [0]:
def make_generator_model():
    inputs = layers.Input(shape=(N_INPUT_PARAMS,))
    x = inputs

    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Reshape((1, 1, 512))(x)

    x = upsample(x, 256, 5)     # 2 x 2 x 256
    x = upsample(x, 128, 5)     # 4 x 4 x 128
    x = upsample(x, 64, 5)      # 8 x 8 x 64
    x = upsample(x, 32, 5)      # 16 x 16 x 32
    x = upsample(x, 16, 5)      # 32 x 32 x 16
    x = upsample(x, 8, 5)       # 64 x 64 x 8

    x = layers.Conv2DTranspose(OUTPUT_CHANNELS, (5, 5), padding='same', use_bias=False, activation='tanh')(x)
    
    print(x.shape)              # 64 x 64 x 4
    assert tuple(x.shape) == (None, SIZE, SIZE, OUTPUT_CHANNELS)

    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

generator = make_generator_model()